# Validating regression models for prediction
Statistical tests are useful for making sure a model is a good fit to the test data, and that all the features are useful to the model. However, to make sure a model has good predictive validity for new data, it is necessary to assess the performance of the model on new datasets.

The procedure is the same as what you learned in the Naive Bayes lesson – the holdout method and cross-validation method are both available. You've already had experience writing code to run these kinds of validation models for Naive Bayes: now you can try it again with linear regression. In this case, your goal is to achieve a model with a consistent R2 and only statistically significant parameters across multiple samples.

We'll use the property crime model you've been working on with, based on the FBI:UCR data. Since your model formulation to date has used the entire New York State 2013 dataset, you'll need to validate it using some of the other crime datasets available at the FBI:UCR website. Options include other states crime rates in 2013 or crime rates in New York State in other years or a combination of these.

# Iterate
Based on the results of your validation test, create a revised model, and then test both old and new models on a new holdout or set of folds.

Include your model(s) and a brief writeup of the reasoning behind the validation method you chose and the changes you made to submit and review with your mentor.

# Load and clean data

In [113]:
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std

%matplotlib inline
sns.set_style('white')

In [114]:
from clean import clean

c2013_df = clean(pd.read_csv("NYC_2013_crime.csv"))
# 2014 data has an extra column "Unnamed: 13" full of NaNs; we'll drop it.
c2014_df = clean(pd.read_csv("NYC_2014_crime.csv").drop("Unnamed: 13", axis=1))

To confirm that the datasets are similar, I'll print out the percentage change in the statistics for each colummn.

In [115]:
print( (c2013_df.describe() - c2014_df.describe()) / c2013_df.describe() )

       Population  violent_crime    Murder   Robbery  agg_assault  prop_crime  \
count   -0.058140      -0.058140 -0.058140 -0.058140    -0.058140   -0.058140   
mean     0.068316       0.061423  0.071381  0.081682     0.088971    0.150452   
std      0.013047       0.032663  0.092162  0.041533     0.050897    0.077943   
min      0.849810            NaN       NaN       NaN          NaN         NaN   
25%      0.123607       0.500000       NaN       NaN     1.000000    0.365385   
50%      0.093161       0.166667       NaN  0.000000     0.250000    0.327354   
75%      0.149335       0.190476       NaN  0.200000     0.150943    0.209858   
max     -0.004867       0.023490  0.047619 -0.015000     0.040230    0.110922   

       Burglary   Larceny  mv_theft     Arson  
count -0.058140 -0.058140 -0.058140 -0.058140  
mean   0.201633  0.141396  0.101632 -0.681107  
std    0.179848  0.049689  0.065357 -0.070309  
min         NaN       NaN       NaN       NaN  
25%    0.333333  0.354839     

Looking at the percentage changes in the means of the columns above, we can see that all of these changes look small and feasible exxcept for the 68% drop in arson.  Given the likelihood that this represents a difference in reporting, we'll drop the arson column.  Otherwise, the data all looks similar, so we'll go ahead and combine the two years for a larger dataset.

In [83]:
c2013_df["Year"] = 2013
c2014_df["Year"] = 2014
crime_df = c2013_df.append(c2014_df)
crime_df.drop("Arson", axis=1, inplace=True)
crime_df.sort_values("City").head()

,City,Population,violent_crime,Murder,Robbery,agg_assault,prop_crime,Burglary,Larceny,mv_theft,Year
0,Adams Village,1861,0,0,0,0,12,2,10,0,2013
0,Adams Village,1851,0,0,0,0,11,1,10,0,2014
1,Addison Town and Village,2568,2,0,1,1,49,1,47,1,2014
1,Addison Town and Village,2577,3,0,0,3,24,3,20,1,2013
2,Afton Village4,820,0,0,0,0,1,0,1,0,2014


# Running the model

In my previous model, I predicted violent crime rates from the population and the specific nonviolent crime statistics of burglary, larceny, and motor vehicle theft.  The model yielded an r-score of just over .8, and relied primarily on motor vehicle theft and secondarily on burglary to generate this prediction.

To begin here, I'll run the model as it was.

In [127]:
# randomly select 30% of rows as CV set
from sklearn.model_selection import train_test_split
y = crime_df["violent_crime"]
X_train, X_cv, y_train, y_cv = train_test_split(crime_df[["Population","Burglary","Larceny","mv_theft"]], y, test_size=0.3)

In [129]:
# run the model repeatedly to generate typical outcome regardless of random data division (because dataset is small)
def regtest(crime_df):
    # randomly select 30% of rows as CV
    y = crime_df["violent_crime"]
    X_train, X_cv, y_train, y_cv = train_test_split(crime_df[["Population","Burglary","Larceny","mv_theft"]], y, test_size=0.3)
    X_train.head()

    from sklearn import linear_model
    rModel = linear_model.LinearRegression()
    rModel.fit(X_train, y_train)
    return rModel.coef_, rModel.score(X_cv, y_cv)

rcoefs = []
rscores = []
for i in range(200):
    coef, score = regtest(crime_df)
    rscores.append(score)
    rcoefs.append(coef)
print(sum(rscores)/len(rscores))
print(sum(rcoefs)/len(rcoefs))

0.8668017641453997
[-1.48583086e-04  4.06881365e-01 -5.61832796e-03  2.01515689e+00]


This R-squared value already represents a roughly .5 improvement on the previous outcomes, simply because I have twice the data to work with.

We can also eliminate population and get basically the same R-value:

In [130]:
# run the model repeatedly to generate typical outcome regardless of random data division (because dataset is small)
def regtest(crime_df):
    # randomly select 30% of rows as CV
    y = crime_df["violent_crime"]
    X_train, X_cv, y_train, y_cv = train_test_split(crime_df[["Burglary","Larceny","mv_theft"]], y, test_size=0.3)
    X_train.head()

    from sklearn import linear_model
    rModel = linear_model.LinearRegression()
    rModel.fit(X_train, y_train)
    return rModel.coef_, rModel.score(X_cv, y_cv)

rcoefs = []
rscores = []
for i in range(200):
    coef, score = regtest(crime_df)
    rscores.append(score)
    rcoefs.append(coef)
print(sum(rscores)/len(rscores))
print(sum(rcoefs)/len(rcoefs))

0.8648139938255555
[ 0.40011015 -0.009714    1.98745989]


Now let's try testing this model with an F-test and the features with individual t-tests. 

In [ ]:
import statsmodels.formula.api as smf
from statsmodels.sandbox.regression.predstd import wls_prediction_std



In [107]:
from sklearn.feature_selection import f_regression, f_classif
for arr in f_regression(X_train, y_train):
    print(arr)
print("**********")
for arr in f_classif(X_train, y_train):
    print(arr)
# Why are these two different???

# http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_classif.html
# http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression

[ 443.01389889 3195.58656698  866.84309624 2996.40436816]
[1.20903013e-070 1.38706327e-217 1.08230379e-110 1.22123047e-211]
**********
[ 29.01086101 274.87736369  63.95856581 212.9793068 ]
[2.05453426e-132 9.31429253e-314 4.21428771e-193 3.22957546e-292]


In [109]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
formula = 'violent_crime ~ Population + Burglary + Larceny + mv_theft'
results = smf.ols(formula, data=crime_df).fit()
print(results.summary())

/usr/local/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


                            OLS Regression Results                            
Dep. Variable:          violent_crime   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     1618.
Date:                Mon, 14 May 2018   Prob (F-statistic):               0.00
Time:                        17:14:54   Log-Likelihood:                -3470.2
No. Observations:                 708   AIC:                             6950.
Df Residuals:                     703   BIC:                             6973.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2202      1.589     -1.397      0.1

In [112]:
crime_df["pop2"] = crime_df["Population"]**.5
crime_df["burg2"] = crime_df["Burglary"]**.5
crime_df["larceny2"] = crime_df["Larceny"]**.5
crime_df["mv2"] = crime_df["mv_theft"]**.5

formula = 'violent_crime ~ pop2 + burg2 + larceny2 + mv2'
results2 = smf.ols(formula, data=crime_df).fit()
print(results2.summary())

                            OLS Regression Results                            
Dep. Variable:          violent_crime   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     555.4
Date:                Mon, 14 May 2018   Prob (F-statistic):          6.53e-216
Time:                        17:40:36   Log-Likelihood:                -3788.0
No. Observations:                 708   AIC:                             7586.
Df Residuals:                     703   BIC:                             7609.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -26.6824      4.017     -6.642      0.0

In [131]:
formula = 'violent_crime ~ Population + Burglary + Larceny + mv_theft'
results3 = smf.ols(formula, data=crime_df).fit()
print(results3.summary())

                            OLS Regression Results                            
Dep. Variable:          violent_crime   R-squared:                       0.902
Model:                            OLS   Adj. R-squared:                  0.901
Method:                 Least Squares   F-statistic:                     1618.
Date:                Tue, 15 May 2018   Prob (F-statistic):               0.00
Time:                        10:33:35   Log-Likelihood:                -3470.2
No. Observations:                 708   AIC:                             6950.
Df Residuals:                     703   BIC:                             6973.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -2.2202      1.589     -1.397      0.1

In [ ]:
'''
What I'm doing now:
1) copy over my Predicting Violent Crime model from 4.4
2) translate it into stats-models
3) evaluate it with f-score and p-vals.  Consider checking resid counts / homoscedacity/ collinearity too (4.2)
4) tweak as necessary
'''